In [1]:
!git clone https://github.com/dcxjn/prompting.git /content/prompting

Cloning into '/content/prompting'...
remote: Enumerating objects: 567, done.
remote: Counting objects: 100% (567/567), done.
remote: Compressing objects: 100% (432/432), done.
remote: Total 567 (delta 179), reused 486 (delta 116), pack-reused 0
Receiving objects: 100% (567/567), 9.29 MiB | 8.78 MiB/s, done.
Resolving deltas: 100% (179/179), done.


In [2]:
import os
os.chdir('/content/prompting')

In [3]:
import sys
sys.path.append('/content/prompting')

In [4]:
!pip install torch>=2.0.0
!pip install tiktoken
!pip install einops
!pip install transformers_stream_generator
!pip install accelerate
!pip install optimum
!pip install auto-gptq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for transformers_stream_generator: filename=transformers_stream_generator-0.0.5-py3-none-any.whl size=12425 sha256=dc392ceb47df8296ce44d90e4395ce871ab6f0fe0dcdb08a1170ab0a4ab48fe3
  Stored in directory: /root/.cache/pip/wheels/95/4a/90/140f7b67d125906f6a165f38aad212ecb4a695ad0d87582437
Successfully built transformers_stream_generator
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvi

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
import torch

from src.utils.image_util import resize_image

In [6]:
def query(inputs: dict) -> dict:

    # Set tokenizer
    tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-VL-Chat-Int4", trust_remote_code=True)

    model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-VL-Chat-Int4", device_map="cuda", trust_remote_code=True).eval()
    # model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-VL-Chat-Int4", device_map="cuda", trust_remote_code=True).eval()

    prompt1 = f"""
    Observe the given image and its details.
    Provide a detailed step-by-step guide on how a human would complete the task of: {inputs["task"]}.
    Link each instruction to an observation in the image in this format: Observation - Instruction.
    """

    query = tokenizer.from_list_format([
        {'image': inputs['image_path']},
        {'text': prompt1},
    ])

    output1, history = model.chat(tokenizer, query=query, history=None)
    print("\n=== OUTPUT 1 ===\n") # for debugging
    print(output1)

    prompt2 = f"""
    Imagine you are in control of a robotic arm with the following commands: {inputs["bot_commands"]}
    Given the human instructions you have generated, provide a guide on how the robot would complete the task.
    """

    output2, history = model.chat(tokenizer, prompt2, history=history)
    print("\n=== OUTPUT 2 ===\n") # for debugging
    print(output2)

    prompt3 = f"""
    By referencing an observation in the image, ensure each instruction is accurate. Do not make assumptions.
    Check that each instruction is logical.
    """

    output3, history = model.chat(tokenizer, prompt3, history=history)

    return {"bot_inst": output3}


In [7]:
# Robot commands available
bot_commands = """
    1. move_to(x, y)
    2. grab(object)
    3. release(object)
    4. push(object)
    5. pull(object)
    6. rotate(angle)
"""

In [8]:
# image_path = input("Enter the path of the image: ")
# image_path = r"images/fridge_lefthandle.jpg"
# image_path = r"images/housedoor_knob_push.jpg"
# image_path = r"images/browndoor_knob_pull.jpg"
# image_path = r"images/labdoor_straighthandle_pull.jpg"
image_path = r"images/bluedoor_knob_push.jpg"
# image_path = r"images/whitetable.jpg"

In [9]:
resize_image(image_path, image_path)

In [10]:
# Define the task to be performed
task = input("Enter the task to be performed: ")

Enter the task to be performed: open the door


In [11]:
result = query(
    {
        "image_path": image_path,
        "task": task,
        "bot_commands": bot_commands,
    }
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

tokenization_qwen.py:   0%|          | 0.00/21.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Qwen/Qwen-VL-Chat-Int4:
- tokenization_qwen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


qwen.tiktoken:   0%|          | 0.00/2.56M [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4481: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some weights of the model checkpoint at Qwen/Qwen-VL-Chat-Int4 were not used when initializing QWenLMHeadModel: ['transformer.h.0.attn.c_proj.bias', 'transformer.h.0.mlp.c_proj.bias', 'transformer.h.0.mlp.w1.bias', 'transformer.h.0.mlp.w2.bias', 'transformer.h.1.attn.c_proj.bias', 'transformer.h.1.mlp.c_proj.bias', 'transformer.h.1.mlp.w1.bias', 'transformer.h.1.mlp.w2.bias', 'transformer.h.10.attn.c_proj.bias', 'transformer.h.10.mlp.c_proj.bias', 'transformer.h.10.mlp.w1.bias', 'transformer.h.10.mlp.w2.bias', 'transformer.h.11.attn.c_proj.bias', 'transformer.h.11.mlp.c_proj.bias', 'transformer.h.11.mlp.w1.bias', 'transformer.h.11.mlp.w2.bias', 'transformer.h.12.attn.c_proj.bias', 'transformer.h.12.mlp.c_proj.bias', 'transformer.h.12.mlp.w1.bias', 'transformer.h.12.mlp.w2.bias', 'transformer.h.13.attn.c_proj.bias', 'transformer.h.13.mlp.c_proj.bias', 'transformer.h.13.mlp.w1.bias', 'transformer.h.13.mlp.w2.bias', 'transformer.h.14.attn.c_proj.bias', 'transformer.h.14.mlp.c_proj.bias', 

generation_config.json:   0%|          | 0.00/221 [00:00<?, ?B/s]


=== OUTPUT 1 ===

Observation - Instruction 1: Stand near the door. Since the image shows a blue door with a doormat in front, it is evident that the door is easily accessible for anyone to open. Instruction 2: Use the doorknob to turn the door handle. The door handle is located near the middle of the door, making it easy to locate and turn. Instruction 3: Pull the door towards you to open it. The door is made of wood and is hinged, which means it can be easily opened by pulling it towards you.

=== OUTPUT 2 ===

Sure, here's a step-by-step guide on how the robotic arm could complete the task of opening the door using the provided human instructions:

1. Move the robotic arm to the position where the door handle is located. The human instructions do not specify the exact position, so the robotic arm would need to be programmed to move to the location of the door handle based on the specific door design and dimensions.

2. Grab the door handle. The human instructions do not specify the

In [13]:
print("\n==========\n")
print(result["bot_inst"])



Observation - Instruction 1: The door is blue and made of wood. Instruction 2: The door handle is located near the middle of the door. Instruction 3: The door is hinged, which means it can be easily opened by pulling it towards you. Instruction 4: The door is made of wood, which means it can be easily pushed and pulled to open and close it. Instruction 5: The door is made of wood, which means it can be easily rotated to open and close it. Instruction 6: The door is made of wood, which means it can be easily grabbed and released to open and close it.
